# Energy-Based Model (EBM) on MNIST
This notebook implements a basic EBM using PyTorch and MNIST.

In [ ]:
# Install and import dependencies
!pip install torch torchvision matplotlib --quiet
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Set up MNIST data loader
transform = transforms.Compose([transforms.ToTensor()])
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(mnist, batch_size=64, shuffle=True)